In [11]:
from keras.models import load_model
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os
import time 

# Violence Detection Model

In [12]:
import winsound
def alarm():
    duration = 1000  # milliseconds
    freq = 440  # Hz
    winsound.Beep(freq, duration)

In [13]:
import numpy as np
import argparse
import pickle
import cv2
import os
import time 
from keras.models import load_model
from collections import deque

cv2.startWindowThread()

print("Loading model ...")
model = load_model('model.h5')
Q = deque(maxlen=128)

vs = cv2.VideoCapture(0)

font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0, 255, size=(100, 3))
writer = None
(W, H) = (None, None)
count = 0     
while True:
    # read the next frame from the file
    (grabbed, frame) = vs.read()

    # if the frame was not grabbed, then we have reached the end
    # of the stream
    if not grabbed:
        break
            
    # if the frame dimensions are empty, grab them
    if W is None or H is None:
        (H, W) = frame.shape[:2]

    # clone the output frame, then convert it from BGR to RGB
    # ordering, resize the frame to a fixed 128x128, and then
    # perform mean subtraction
 
    output = frame.copy()
           
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (128, 128)).astype("float32")
    frame = frame.reshape(128, 128, 3) / 255

    # make predictions on the frame and then update the predictions
    # queue
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    # print("preds",preds)
    Q.append(preds)

    # perform prediction averaging over the current history of
    # previous predictions
    results = np.array(Q).mean(axis=0)
    i = (preds > 0.50)[0]
    label = i

    text_color = (0, 255, 0) # default : green BGR

    if label: # Violence prob
        text_color = (0, 0, 255)# red
        #calling alarm fucntion
        alarm()

    else:
        text_color = (0, 255, 0)

    text = "Violence: {}".format(label)
    FONT = cv2.FONT_HERSHEY_SIMPLEX 

    cv2.putText(output, text, (35, 50), FONT,1.25, text_color, 3) 

    # check if the video writer is None
    if writer is None:
        # initialize our video writer
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter("violence_output.avi", fourcc, 30,(W, H), True)

    # write the output frame to disk
    writer.write(output)

    # show the output image
    # cv2_imshow(output)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('p'):
        break
# release the file pointersq
print("[INFO] cleaning up...")
writer.release()
vs.release()
cv2.destroyAllWindows()

Loading model ...
1/1 [==============================] - 0s 31ms/step
[INFO] cleaning up...


In [15]:
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')

classes = []
with open("coco.names", "r") as f:
    classes = f.read().splitlines()

cap = cv2.VideoCapture('violence_output.avi')
font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0, 255, size=(100, 3))
(height, width)=(None, None)
writer = None
while True:
    _, img = cap.read()
    # height, width, _ = img.shape
    
    if not _:
      break
            
    # if the frame dimensions are empty, grab them
    if width is None or height is None:
      (height, width) = img.shape[:2]

    out = img.copy()

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (128, 128)).astype("float32")
    img = img.reshape(128, 128, 3) / 255

    blob = cv2.dnn.blobFromImage(out, 1/255, (416, 416), (0,0,0), swapRB=True, crop=False)
    net.setInput(blob)
    output_layers_names = net.getUnconnectedOutLayersNames()
    layerOutputs = net.forward(output_layers_names)

    boxes = []
    confidences = []
    class_ids = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.2:
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                x = int(center_x - w/2)
                y = int(center_y - h/2)

                boxes.append([x, y, w, h])
                confidences.append((float(confidence)))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.2, 0.4)

    if len(indexes)>0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i],2))
            color = colors[i]
            cv2.rectangle(out, (x,y), (x+w, y+h), color, 2)
            cv2.putText(out, label + " " + confidence, (x, y+20), font, 2, (255,255,255), 2)

    # check if the video writer is None
    if writer is None:
      # initialize our video writer
      fourcc = cv2.VideoWriter_fourcc(*"MJPG")
      writer = cv2.VideoWriter("violence_output_people_detection_1.avi", fourcc, 30,(width, height), True)
      #writer = cv2.VideoWriter("violence_output_people_detection_1.mp4", fourcc, 30,(width, height), True)

    # write the output frame to disk
    writer.write(out)

    #cv2_imshow(out)
    key = cv2.waitKey(1)
    if key==27:
        break

cap.release()
cv2.destroyAllWindows()

In [33]:
from IPython.display import HTML
from base64 import b64encode
vid1 = open('violence_output_people_detection_1.avi','rb').read()
data_url = "video/mp4|;base64," + b64encode(vid1).decode()
HTML("""
<video width=600 controls>
      <source src="%s" type="video/mp4">
</video>
"""%data_url)

In [34]:
from IPython.display import Video
Video.from_file("violence_output_people_detection_1.avi")

AttributeError: type object 'Video' has no attribute 'from_file'

In [24]:
from IPython.display import HTML

HTML("""
    <video alt="test" controls>
        <source src="violence_output_people_detection_1.avi" type="video/avi">
    </video>
""")